## 对模型超参数进行调优(调参)：

上述的方法主要是对模型算法本身的优化改进，而找到最佳参数的问题本质上属于最优化的内容，因为从一个参数集合中找到最佳的值本身就是最优化的任务之一。

- 参数与超参数：
    
参数w是我们通过设定某一个具体的$\lambda$后使用类似于最小二乘法、梯度下降法等方式优化出来的，我们总是设定了$\lambda$是多少后才优化出来的参数w。因此，类似于参数w一样，使用最小二乘法或者梯度下降法等最优化算法优化出来的数我们称为参数，类似于$\lambda$一样，我们无法使用最小二乘法或者梯度下降法等最优化算法优化出来的数我们称为超参数。

模型参数是模型内部的配置变量，其值可以根据数据进行估计。

- 进行预测时需要参数。                     
- 它参数定义了可使用的模型。                        
- 参数是从数据估计或获悉的。                       
- 参数通常不由编程者手动设置。                     
- 参数通常被保存为学习模型的一部分。                      
- 参数是机器学习算法的关键，它们通常由过去的训练数据中总结得出 。                          

模型超参数是模型外部的配置，其值无法从数据中估计。
      
- 超参数通常用于帮助估计模型参数。
- 超参数通常由人工指定。
- 超参数通常可以使用启发式设置。
- 超参数经常被调整为给定的预测建模问题。

### 网格搜索GridSearchCV()

```python

class sklearn.model_selection.GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)

```

```python
#网格搜索结合管道

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA, NMF
from sklearn.feature_selection import SelectKBest, chi2

 
```

网格搜索的思想，比如你有2个超参数需要去选择，那你就把所有的超参数选择列出来分别做排列组合。举个例子：$\lambda = 0.01,0.1,1.0$和$\alpha = 0.01,0.1,1.0$,你可以做一个排列组合，即：{[0.01,0.01],[0.01,0.1],[0.01,1],[0.1,0.01],[0.1,0.1],[0.1,1.0],[1,0.01],[1,0.1],[1,1]}  ，然后针对每组超参数分别建立一个模型，然后选择测试误差最小的那组超参数。换句话说，我们需要从超参数空间中寻找最优的超参数，很像一个网格中找到一个最优的节点，因此叫网格搜索

## 随机搜索 RandomizedSearchCV():
```python
class sklearn.model_selection.RandomizedSearchCV(estimator, param_distributions, *, n_iter=10, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', random_state=None, error_score=nan, return_train_score=False)[source]¶
```
那就是使用随机搜索的方式，这种方式不仅仅高校，而且实验证明，随机搜索法结果比稀疏化网格法稍好(有时候也会极差，需要权衡)。参数的随机搜索中的每个参数都是从可能的参数值的分布中采样的。与网格搜索相比，这有两个主要优点：
- 可以独立于参数数量和可能的值来选择计算成本。
- 添加不影响性能的参数不会降低效率。

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")
import seaborn as sns

Missing colon in file '/Users/ousei/.matplotlib/matplotlibrc', line 1 ('{\\rtf1\\ansi\\ansicpg936\\cocoartf1671\\cocoasubrtf200')
Missing colon in file '/Users/ousei/.matplotlib/matplotlibrc', line 2 ('{\\fonttbl\\f0\\fnil\\fcharset0 Menlo-Regular;}')
Missing colon in file '/Users/ousei/.matplotlib/matplotlibrc', line 3 ('{\\colortbl;\\red255\\green255\\blue255;\\red0\\green0\\blue0;\\red255\\green255\\blue255;}')
Missing colon in file '/Users/ousei/.matplotlib/matplotlibrc', line 4 ('{\\*\\expandedcolortbl;;\\cssrgb\\c0\\c0\\c0;\\cssrgb\\c100000\\c100000\\c100000;}')
Missing colon in file '/Users/ousei/.matplotlib/matplotlibrc', line 5 ('\\paperw11900\\paperh16840\\margl1440\\margr1440\\vieww10800\\viewh8400\\viewkind0')
Missing colon in file '/Users/ousei/.matplotlib/matplotlibrc', line 6 ('\\deftab720')
Missing colon in file '/Users/ousei/.matplotlib/matplotlibrc', line 7 ('\\pard\\pardeftab720\\sl260\\partightenfactor0')
Missing colon in file '/Users/ousei/.matplotlib/matplotlibrc'

In [1]:
'''
使用SVR，结合管道进行评价：
'''
from sklearn.svm import SVR
#引入管道简化学习流程
from sklearn.pipeline import make_pipeline
#由于SVR基于距离计算，引入对数据进行标准化的类
from sklearn.preprocessing import StandardScaler
#引入网格搜索调优
from sklearn.model_selection import GridSearchCV
#引入K折交叉验证
from sklearn.model_selection import cross_val_score

In [4]:
from sklearn import datasets
boston = datasets.load_boston()       #返回一个类似字典的类
X = boston.data
y = boston.target
features = boston.feature_names
boston_data = pd.DataFrame(X,columns=features)
boston_data["Price"] = y
boston_data.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,Price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [5]:
pipe_SVR = make_pipeline(StandardScaler(),SVR())
score1 = cross_val_score(estimator=pipe_SVR,X=X,y=y,scoring='r2',cv=10)
print("CV accuracy: %.3f +/- %.3f" % ((np.mean(score1)),np.std(score1)))

CV accuracy: 0.187 +/- 0.649


In [9]:
#使用网络搜索来对SVR调参
from sklearn.pipeline import Pipeline
pipe_svr = Pipeline([("StandardScaler",StandardScaler()),("svr",SVR())])
param_range = [0.0002,0.002,0.02,0.2,2.0,20.0,200.0,2000.0]
#双下划线
param_grid = [{"svr__C":param_range,"svr__kernel":["linear"]},  
{"svr__C":param_range,"svr__gamma":param_range,"svr__kernel":["rbf"]}]
gs = GridSearchCV(estimator=pipe_svr,param_grid = param_grid,
scoring = 'r2',cv = 10)
gs = gs.fit(X,y)
print("网格搜索最优得分：",gs.best_score_)
print("网格搜索最优参数组合：\n",gs.best_params_)

网格搜索最优得分： 0.5948781524691318
网格搜索最优参数组合：
 {'svr__C': 200.0, 'svr__gamma': 0.002, 'svr__kernel': 'rbf'}
